# Custom Legend with `manualKey` Option

In Lets-Plot-Kotlin, as in ggplot2, legends are automatically generated based on the aesthetic mappings in the plot. 

Sometimes, however, this automatic generation doesn't provide the precise control needed for complex visualizations. 

The new `manualKey` option in plot layer addresses this limitation by allowing you to create custom legend entries specific to individual plot layers.

The `manualKey` option can be used in two ways:

- Simple case: pass a string to serve as the label for the legend entry.
- Advanced case: use the `layerKey()` function for more detailed customization:

    - `label` - text for the legend element
    - `group` - key used to group elements in the legend
    - `index` - position of the element within its legend group
    - `alpha`, `color`, `fill`, `shape`, `size`, `width`, `height`, `linetype`, `stroke` - aesthetic parameters to be applied in the legend

In [1]:
%useLatestDescriptors
%use lets-plot
%use dataframe

In [2]:
LetsPlot.getInfo()

Lets-Plot Kotlin API v.0.0.0-SNAPSHOT. Frontend: Notebook with dynamically loaded JS. Lets-Plot JS v.4.4.0.

In [3]:
val mpgDf = DataFrame.readCSV("https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/data/mpg.csv")
val mpgData = mpgDf.toMap()
mpgDf.head(3)

DataFrame: rowsCount = 3, columnsCount = 12 
 untitled manufacturer model displ year cyl trans drv cty hwy fl class 1 audi a4 1,800000 1999 4 auto(l5) f 18 29 p compact 2 audi a4 1,800000 1999 4 manual(m5) f 21 29 p compact 3 audi a4 2,000000 2008 4 manual(m6) f 20 31 p compact

In [4]:
letsPlot(mpgData) { x = "displ"; y = "hwy" } + 
    geomPoint(color = "#878787") +
    geomSmooth(method = "loess", se = false, color = "#2166ac") +
    geomSmooth(method = "lm", se = false, color = "#b2182b")

#### 1. Simple Custom Legend

In [5]:
letsPlot(mpgData) { x = "displ"; y = "hwy" } + 
    geomPoint(color = "#878787", 
              manualKey = "Observed Data") +                         // <-- label for the legend entry
    geomSmooth(method = "loess", se = false, color = "#2166ac", 
               manualKey = "LOESS Trend Line") +
    geomSmooth(method = "lm", se = false, color = "#b2182b",  
               manualKey = "Linear Model Trend Line") +
    ggsize(800, 400)

#### 2.  The `layer_key()` Function

All constants for the visual representation of the legend key are inherited from the geometry. Use the `layerKey()` function to override these values. You can also specify the legend group in which the element is displayed and the position of the element within the legend group.

In [6]:
val p = letsPlot(mpgData) { x = "displ"; y = "hwy" } + 
    geomPoint(color = "#878787",
              manualKey = layerKey("Observed Data",                  // <-- label for the legend entry
                                   color = "pen", size = 8)          // <-- override aesthetics
    ) +
    geomSmooth(method = "loess", se = false, color = "#2166ac",
               manualKey = layerKey("LOESS", 
                                    group = "Methods",               // <-- assign a group (note the same group in the layer below)
                                    index = 1, size = 4)) +
    geomSmooth(method = "lm", se = false, color = "#b2182b",
               manualKey = layerKey("LM", 
                                    group = "Methods", index = 0, size = 4)) +
    ggsize(800, 400)
p

#### 3. Change Title of Custom Legends

In [7]:
p + labsAlt(
    "manual" to "Scatter Plot",  // <-- "manual" is a default 'key' referring to a custom legend
    "Methods" to "Trend Lines"   // <-- "Methods" is the group name referring to the second custom legend
)

#### 4. Compound Legend Entries in Custom Legend

In [8]:
val mpg1999 = mpgDf.filter { it["year"] == 1999 }.toMap()
val mpg2008 = mpgDf.filter { it["year"] == 2008 }.toMap()

letsPlot(mpgData) { x = asDiscrete("class", orderBy = "..y.."); y = "hwy" } +
    // 1999
    statSummary(data = mpg1999, fn = "mean",
        geom = Geom.line(), size = 2.0, color = "#D87093",
        manualKey = "1999") +                     // <-- Label for the legend entry
    
    statSummary(data = mpg1999, fn = "mean",
        geom = Geom.point(), size = 5.0, shape = 22,
        fill = "#D87093", color = "paper",
        manualKey = layerKey("1999",              // <-- Using the same label as above results in a composite entry in the legend
                             size = 3.0)) +
    // 2008
    statSummary(data = mpg2008, fn = "mean", 
        geom = Geom.line(), size = 2.0, color = "#708090",
        manualKey = "2008") +
    
    statSummary(data = mpg2008, fn = "mean",
        geom = Geom.point(), size = 7.0, shape = 23, fill = "#708090", color = "paper",
        manualKey = layerKey(label = "2008", size = 5.0)) +
    
    guides(manual = guideLegend(
        stroke = 0,                    // <-- Remove stroke in the "manual" legend
        size = 1.0)) +                 // <-- Update size if not specified otherwise in the layer itself
    
    ggsize(800, 400) +
    theme(legendBackground = elementRect(size = 1.0, linetype = "dotted"))
        .legendDirectionHorizontal()
        .legendPosition(0.9, 0.95)
        .legendJustification(1.0, 1.0)
        